 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/bioinformaticslab/PolyA%20analysis.ipynb#Error-probability-of-the-base-based-on-phred-score" data-toc-modified-id="Error-probability-of-the-base-based-on-phred-score-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Error probability of the base based on phred score</a></span></li><li><span><a href="http://localhost:8888/notebooks/bioinformaticslab/PolyA%20analysis.ipynb#Markov-chain-after-PolyA-analysis" data-toc-modified-id="Markov-chain-after-PolyA-analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Markov chain after PolyA analysis</a></span></li></ul></div>

# Error probability of the base based on phred score 

In [109]:
def base_calling_error_probabilities(ascii_code):
    assert isinstance(ascii_code, str)
    assert len(ascii_code) == 1
    q = ord(ascii_code) - 33
    p = round(10 ** (-q / 10), 5)
    return p

In [110]:
def disrupted_poly_A(read,min_len_poly_a=8, min_len_poly_a2 = 5):
    """
    find poly A tail in a read that is disrupted only by one base other than 'A'
    
    """
    import re
    
    min_len_poly_a = str(min_len_poly_a)
    min_len_poly_a2 = str(min_len_poly_a2)
    pattern = "[A]{" + min_len_poly_a + ",}([^A])[A]{" + min_len_poly_a2 + ",}"
    m = re.search(pattern, read)  
    if m:
        return m.span(1)[0]
    return ''

In [111]:
from collections import namedtuple
phred_data = namedtuple('PhredData', 'ReadNumber PositionOfBase ErrorProb')
n=160000
result = []
path_to_file = '/Users/dariapodorskaja/Desktop/Lab/Lane_7_NoIndex_L007_R1_001.fastq'
with open(path_to_file, 'r') as f:   # f = open(path_to_file, 'r') # f is a variable with a file
    for line_number in range(n):
        if line_number %4 ==1:
            s = f.readline().replace('\n', '')
            position = disrupted_poly_A(s)
            read_number = line_number
        elif line_number %4 ==3 and position != '':
            s = f.readline().replace('\n', '')
            p = base_calling_error_probabilities(s[position])
            result.append(phred_data(read_number, position, p))
        else:
            next(f)

In [112]:
threshold = 0.5 #min error probability
print(sum([1 for rn, pos, prob in result if prob > threshold]))

for r in result:
    rn, pos, prob = r
    if prob > threshold:
        print(r)


111
PhredData(ReadNumber=285, PositionOfBase=20, ErrorProb=0.63096)
PhredData(ReadNumber=1805, PositionOfBase=33, ErrorProb=0.63096)
PhredData(ReadNumber=1909, PositionOfBase=20, ErrorProb=0.63096)
PhredData(ReadNumber=2229, PositionOfBase=20, ErrorProb=0.63096)
PhredData(ReadNumber=4109, PositionOfBase=27, ErrorProb=0.63096)
PhredData(ReadNumber=4373, PositionOfBase=49, ErrorProb=0.63096)
PhredData(ReadNumber=4669, PositionOfBase=20, ErrorProb=0.63096)
PhredData(ReadNumber=4973, PositionOfBase=20, ErrorProb=0.63096)
PhredData(ReadNumber=5113, PositionOfBase=20, ErrorProb=0.63096)
PhredData(ReadNumber=6625, PositionOfBase=20, ErrorProb=0.63096)
PhredData(ReadNumber=6685, PositionOfBase=20, ErrorProb=0.63096)
PhredData(ReadNumber=7249, PositionOfBase=49, ErrorProb=0.63096)
PhredData(ReadNumber=8361, PositionOfBase=43, ErrorProb=0.63096)
PhredData(ReadNumber=13373, PositionOfBase=27, ErrorProb=0.63096)
PhredData(ReadNumber=13685, PositionOfBase=29, ErrorProb=0.63096)
PhredData(ReadNumber

# Markov chain after PolyA analysis

In [113]:
# count dna_reads in a file
# open file
path_to_file = '/Users/dariapodorskaja/Desktop/Lab/Lane_7_NoIndex_L007_R1_001.fastq'
dna_reads = []
lines_count = 160000  # number of lines in a file
#Result = []

# make lists of dna reads
with open(path_to_file, 'r') as f:   # f = open(path_to_file, 'r') # f is a variable with a file
    for current_line in range(lines_count):
        if current_line % 4 == 1:  # pick lines with DNA sequences
            s = f.readline()
            s = s.replace('\n', '')
            dna_reads.append(s)
        else:
            next(f)

In [114]:
def after_polyatail(read, next_n_bases=10, min_len_poly_a=8):
    """
    if a poly A tail length is grater than min_len_poly_a
    go to the end of the tail
    and return next n bases
    """
    assert isinstance(read, str)
    counter_a = 0
    bases_after_poly_a = str()
    for index, base in enumerate(read):
        if base == 'A':
            counter_a += 1
        else:
            if counter_a >= min_len_poly_a:
                bases_after_poly_a = read[index:index + next_n_bases]
            counter_a = 0
    return bases_after_poly_a

In [115]:
reads=[]
for dna in dna_reads:
    reads.append(after_polyatail(dna, 4, 3))

In [116]:
from itertools import product
alphabet = 'ACTGN'
# let's do all possible transisions
trans = [i+j for i, j in product(alphabet, alphabet)]  # possible transitions

In [117]:
# now lets do a dict where all initial values are 0
d = {k: 0 for k in trans}
print(d)

{'AA': 0, 'AC': 0, 'AT': 0, 'AG': 0, 'AN': 0, 'CA': 0, 'CC': 0, 'CT': 0, 'CG': 0, 'CN': 0, 'TA': 0, 'TC': 0, 'TT': 0, 'TG': 0, 'TN': 0, 'GA': 0, 'GC': 0, 'GT': 0, 'GG': 0, 'GN': 0, 'NA': 0, 'NC': 0, 'NT': 0, 'NG': 0, 'NN': 0}


In [118]:
# always take next 2 letters
result= []
for read in dna_reads:
    for i in range(len(read)):
        result.append((read[i: i+2]))

In [119]:
result[0:10]

['GC', 'CA', 'AT', 'TT', 'TG', 'GG', 'GA', 'AA', 'AA', 'AA']

In [120]:
# now combine this 2 letters with dict
# that means if we see 2 letters, add +1 to coresponding enty in the dict
for two_letters in result:
    if two_letters in d.keys():
        d[two_letters] += 1

In [121]:
print(d)

{'AA': 592392, 'AC': 161284, 'AT': 94862, 'AG': 169939, 'AN': 92, 'CA': 180503, 'CC': 103086, 'CT': 109070, 'CG': 95283, 'CN': 70, 'TA': 58406, 'TC': 132117, 'TT': 94820, 'TG': 93535, 'TN': 33, 'GA': 189049, 'GC': 91599, 'GT': 81500, 'GG': 112065, 'GN': 48, 'NA': 71, 'NC': 44, 'NT': 22, 'NG': 34, 'NN': 76}


In [122]:
total_transitions_to_a = 0
total_transitions_to_c = 0
total_transitions_to_t = 0
total_transitions_to_g = 0
total_transitions_to_n = 0

for k in d.keys():
    if k.startswith('A'):
        total_transitions_to_a += d[k]
    elif k.startswith('C'):
        total_transitions_to_c += d[k]
    elif k.startswith('T'):
        total_transitions_to_t += d[k]
    elif k.startswith('G'):
        total_transitions_to_g += d[k]
    elif k.startswith('N'):
        total_transitions_to_n += d[k]
    else:
        print("we missed a key!")
        

In [123]:
a_prob = []
c_prob = []
t_prob = []
g_prob = []
n_prob = []

for k in trans:
    if k.startswith('A'):
        a_prob.append((d[k] / total_transitions_to_a))
    elif k.startswith('C'):
        c_prob.append((d[k] / total_transitions_to_c))
    elif k.startswith('T'):
        t_prob.append((d[k] / total_transitions_to_t))
    elif k.startswith('G'):
        g_prob.append((d[k] / total_transitions_to_g))
    elif k.startswith('N'):
        n_prob.append((d[k] / total_transitions_to_n))
    else:
        print("we missed a key!")

In [124]:
matrix = [[0 for i in range(len(alphabet))] for i in range(len(alphabet))]
for i in range(len(alphabet)):
    matrix [0][i] = round(a_prob[i], 2)
    matrix [1][i] = round(c_prob[i], 2)
    matrix [2][i] = round(t_prob[i], 2)
    matrix [3][i] = round(g_prob[i], 2)
    matrix [4][i] = round(n_prob[i], 2)
    
matrix

[[0.58, 0.16, 0.09, 0.17, 0.0],
 [0.37, 0.21, 0.22, 0.2, 0.0],
 [0.15, 0.35, 0.25, 0.25, 0.0],
 [0.4, 0.19, 0.17, 0.24, 0.0],
 [0.29, 0.18, 0.09, 0.14, 0.31]]

In [125]:
import pandas as pd
alphabet = ['A', 'C', 'T', 'G', 'N']
df = pd.DataFrame(matrix, index=alphabet, columns=alphabet)
df

,A,C,T,G,N
A,0.58,0.16,0.09,0.17,0.00
C,0.37,0.21,0.22,0.20,0.00
T,0.15,0.35,0.25,0.25,0.00
G,0.40,0.19,0.17,0.24,0.00
N,0.29,0.18,0.09,0.14,0.31
